# Imports

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Constants

In [2]:
torch.manual_seed(1337)
training_style = "scale"
if training_style == "scale":
    block_size = 64
    batch_size = 512
    max_iters = 5000
    eval_interval = 500
    learning_rate = 5e-4
    eval_iters = 200
    n_embd = 384
    n_layer = 6 # Number of block layers
    n_head = 6
    dropout = 0.2
else:
    block_size = 64
    batch_size = 256
    split_pct = 0.9
    max_iters = 5000
    eval_interval = 500
    learning_rate = 1e-3
    eval_iters = 200
    n_embd = 64
    n_layer = 2 # Number of block layers
    n_head = 2
    dropout = 0.2

In [3]:
# Set GPU device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(device)

cuda


# Dataloader

In [4]:
chars = ['\\', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '.', '=', 'e']
vocab_size = len(chars)
print("Number of tokens: {}".format(vocab_size))

Number of tokens: 16


In [5]:
encoder = { ch:i for i,ch in enumerate(chars) }
decoder = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [encoder[c] if c in chars else print (s) for c in s]
decode = lambda l: ''.join([decoder[i] for i in l])

In [6]:
def get_batch(batch_size):
    numbers = torch.randn((batch_size+4, 2))
    #numbers = torch.randint(0, 1000000, (batch_size+4, 2))
    data = ""
    ix = []
    for idx in range(numbers.size(0) - 1):
        eqn = "{}+{}={}\\".format(numbers[idx, 0], numbers[idx, 1], str((numbers[idx, 0]+numbers[idx, 1]).item())[::-1])
        if idx > 2 and idx < (numbers.size()[0]):
            ix.append(torch.randint(len(data) + eqn.index("=") + 1, len(data) + len(eqn), (1,)).item())
        data = data + eqn
    data = torch.tensor(encode(data))
    x = torch.stack([data[i-block_size:i] for i in ix])
    # y here is the target - the next token for any given point in x
    y = torch.stack([data[i-block_size+1:i+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [7]:
x, y = get_batch(batch_size)
for ix in range(0, len(x), 50):
    print(decode(x[ix].tolist()))
    print(decode(y[ix].tolist()))

944=3657731772572156.0\0.954513669013977+0.06434852629899979=140
44=3657731772572156.0\0.954513669013977+0.06434852629899979=1407
814356013.0-\0.29395735263824463+1.3849540948867798=442052574411
14356013.0-\0.29395735263824463+1.3849540948867798=4420525744119
38621402.2-\-0.08499836921691895+0.11854202300310135=40428168735
8621402.2-\-0.08499836921691895+0.11854202300310135=404281687356
=8596004569456492.1-\-0.7583796381950378+0.06953269243240356=343
8596004569456492.1-\-0.7583796381950378+0.06953269243240356=3436
5358474038905979.0-\0.16433323919773102+-1.35895836353302=205463
358474038905979.0-\0.16433323919773102+-1.35895836353302=2054632
8=110032142209444.0-\-1.7030764818191528+0.8709253668785095=3346
=110032142209444.0-\-1.7030764818191528+0.8709253668785095=33460


# Model

In [8]:
# Obtains average loss over multiple batches
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, T = get_batch(batch_size)
            logits, loss = m(X, T)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [9]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # (C, 3C)
        self.register_buffer('tril', torch.tril(torch.ones(1, 1, block_size, block_size)))
        self.dropout1 = nn.Dropout(dropout)
        self.proj = nn.Linear(n_embd, n_embd) # (C, C)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        # x has shape (B,T,C) - because of initial embeddings
        B, T, C = x.shape

        q, k, v = self.c_attn(x).split(n_embd, dim=2) # (B, T, C) @ (C, 3C) --> (B, T, 3C) splits -> (B, T, C)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)

        # affinities
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, nh, T, hs) @ (B, nh, hs, T) ---> (B,nh,T,T)
        wei = wei.masked_fill(self.tril[:, :, :T, :T] == 0, float('-inf')) # (B, nh, T, T)
        wei = F.softmax(wei, dim=-1) # (B, nh, T, T)
        wei = self.dropout1(wei) # (B, nh, T, T)
        out = wei @ v # (B, nh, T, T) @ (B, nh, T, hs) --> (B, nh, T, hs)
        out = out.transpose(1, 2).contiguous().view(B,T,C) # (B, T, nh, hs) -> (B, T, C)
        out = self.dropout2(self.proj(out)) # (B, T, C) @ (C, C) --> (B, T, C)
        return out

In [10]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Projection layer
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [11]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [12]:
class GptModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tkn_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tkn_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)#, ignore_index=encode("\\")[0])
            
        return logits, loss

    def generate(self, idx):
        stop_idx = encode("\\")[0]
        # idx is (B, T) array of indices in the current context
        while idx[0, -1].item() != stop_idx:
            # crop idx to last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # )B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [13]:
m = GptModel()
m.to(device)

GptModel(
  (token_embedding_table): Embedding(16, 64)
  (position_embedding_table): Embedding(64, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=64, out_features=192, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=64, out_features=192, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)

In [14]:
# Number of parameters in model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.105872 M parameters


# Training

In [15]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [16]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch(batch_size)
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.9260, val loss 2.9257
step 500: train loss 2.0617, val loss 2.0619
step 1000: train loss 2.0576, val loss 2.0577


# Inference

In [ ]:
numbers = torch.randn((10,2))
#numbers = torch.randint(0, 1000000, (10, 2))
data = encode("\\".join(["{}+{}={}".format(numbers[ix, 0], numbers[ix, 1], numbers[ix, 0] + numbers[ix, 1]) for ix in range(10)]))
eqn = torch.randn((1, 2))
val1 = eqn[0, 0].item()
val2 = eqn[0, 1].item()
data = data + encode("\\{}+{}=".format(val1, val2))
data = torch.tensor(data).unsqueeze(0).to(device)
output = decode(m.generate(data[:, -block_size:])[0].tolist())
print("Equation: {}".format(output[output.rindex("\\", 0, -2)+1:output.rindex("=")+1]))
print("Prediction: {}".format(output[output.rindex("=")+1:-1][::-1]))
print("Result: {}".format(val1 + val2))

Equation: 124325+59940=
Prediction: 1.1763426718988324
Result: 184265


# Save the model

In [ ]:
#import time
#torch.save(m.state_dict(), './models/addition_calculator_{}.pt'.format(time.strftime("%Y-%m-%d-%H:%M")))